<a href="https://colab.research.google.com/github/titapag/HBPSHPO/blob/main/HBPSHPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pyswarm

In [ ]:
pip install scikit-optimize

In [ ]:
from pyswarm import pso

In [ ]:
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
from operator import add
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
import matplotlib.pyplot as plt 
import re

In [ ]:
from skopt.space import Categorical, Real
from skopt.utils import use_named_args
from skopt import gp_minimize

In [ ]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [ ]:
def hasLSTM(model):
	firstlayer=model.layers[0].name[:4]
	if(firstlayer=='lstm'):
		answer=True
	else:
		answer=False
	return answer	

In [ ]:
def hasCONV1D(model):
	firstlayer=model.layers[0].name[:6]
	if(firstlayer=='conv1d'):
		answer=True
	else:
		answer=False
	return answer	

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		if out_end_ix > len(sequences):
			break
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
def metrics(model, X_test, y_test, scaler):
	prediction=model.predict(X_test)	
	mse = mean_squared_error(y_test, prediction)
	mae = mean_absolute_error(y_test, prediction)
	y_test=scaler.inverse_transform(y_test)
	prediction=scaler.inverse_transform(prediction)
	mse_ram = mean_squared_error(y_test[:,10], prediction[:,10])
	mae_ram = mean_absolute_error(y_test[:,10], prediction[:,10])
	mse_cpu = mean_squared_error(y_test[:,4], prediction[:,4])
	mae_cpu = mean_absolute_error(y_test[:,4], prediction[:,4])
	return mse, mae, mse_ram, mae_ram, mse_cpu, mae_cpu

In [ ]:
def inference(model, X):
	i=random.randrange(len(X))	
	startS=time.time()	
	prediction=model.predict(X[[i]])	
	timeS=time.time()-startS
	i=random.randrange(len(X)-100)
	startB=time.time()
	prediction=model.predict(X[i:i+101])
	timeB=time.time()-startB
	return timeS, timeB

In [ ]:
def data_preparation(datasetfile):
  dataframe = pd.read_csv(datasetfile, engine='python')
  dataset = dataframe.values
  dataset = dataset.astype('float32')
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaler.fit(dataset)
  dataset = scaler.transform(dataset)
  return dataset, scaler

In [ ]:
def remove_dupes(i, o):
	for j in range(len(i)):
		for k in range(len(i)):
			if (i[j]==i[k] and j!=k):
				if (o[j]>o[k]):
					i.pop(j)
					o.pop(j)
					return remove_dupes(i, o)
				else:
					i.pop(k)
					o.pop(k)
					return remove_dupes(i, o)
	return i, o

In [ ]:
def train_test(model, dataset, lookback):
  if hasLSTM(model) or hasCONV1D(model):
    dataset,datasetY=split_sequences(dataset,lookback,1) 
    datasetY=datasetY.reshape(datasetY.shape[0],datasetY.shape[2]) 
  else:
    dataset,datasetY=split_sequences(dataset,1,1)  
    dataset=dataset.reshape(dataset.shape[0],dataset.shape[2])
    datasetY=datasetY.reshape(datasetY.shape[0],datasetY.shape[2])  
  train_size = int(len(dataset) * 0.67)
  test_size = len(dataset) - train_size
  trainX, testX = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
  trainY, testY = datasetY[0:train_size,:], datasetY[train_size:len(dataset),:] 
  return trainX, testX, trainY, testY

In [ ]:
def getOptimizer(x4, optimizer):
  learning_rate = x4
  if optimizer =='RMSprop':
    optimizer = tensorflow.keras.optimizers.RMSprop(learning_rate=learning_rate)
  elif optimizer =='Adam':
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)
  elif optimizer =='SGD':
    optimizer = tensorflow.keras.optimizers.SGD(learning_rate=learning_rate)
  elif optimizer =='Adagrad':
    optimizer = tensorflow.keras.optimizers.Adagrad(learning_rate=learning_rate)
  elif optimizer =='Adadelta':
    optimizer = tensorflow.keras.optimizers.Adadelta(learning_rate=learning_rate)
  elif optimizer =='Adamax':
    optimizer = tensorflow.keras.optimizers.Adamax(learning_rate=learning_rate)
  else:
    optimizer = tensorflow.keras.optimizers.Nadam(learning_rate=learning_rate)

  return optimizer

In [ ]:
def build_model(activationFunction, optimizer):
  global dataset, defineModel, lookback, typeOfModel, neurons, numOfLayers, dropout, noOfLSTM_CONV, pool_size
  
  if defineModel == False:
    defineModel = True
    models = ['with lstm', 'with conv']
    typeOfModel = random.choice(models)

  model = Sequential()
  neurons = int(round(x0))
  numOfLayers = int(round(x1))
  dropout = truncate(x3,1)
  lookback = int(round(x2))
  optimization = getOptimizer(x4, optimizer)
  noOfLSTM_CONV = int(round(x7))
  pool_size = int(round(x8))
  input_shape=(lookback,dataset.shape[1])
  
  if typeOfModel == 'with lstm': 
    if noOfLSTM_CONV == 1:
      model.add(LSTM(neurons, input_shape=(lookback, dataset.shape[1]), activation=activationFunction, 
                     recurrent_activation='sigmoid', implementation=2))
    elif noOfLSTM_CONV == 2:
      model.add(LSTM(neurons, input_shape=(lookback, dataset.shape[1]), activation=activationFunction, recurrent_activation='sigmoid', implementation=2, return_sequences=True))
      model.add(LSTM(neurons, activation=activationFunction, recurrent_activation='sigmoid', implementation=2)) 
  elif typeOfModel == 'with conv':
    for i in range(1,noOfLSTM_CONV+1):
      model.add(Conv1D(filters=neurons, kernel_size=2, padding='same', activation=activationFunction, input_shape=(lookback,dataset.shape[1])))
      model.add(MaxPooling1D(pool_size=pool_size, padding='same'))
      if i == 2:
        model.add(Conv1D(filters=neurons, kernel_size=2, padding='same', activation=activationFunction))
        model.add(MaxPooling1D(pool_size=pool_size, padding='same'))
    model.add(Flatten())

  units = neurons
  for i in range(1,numOfLayers+1):
      units = int(round(units/2))
      if units > 0:    
        model.add(Dense(units, activation=activationFunction))
        model.add(Dropout(dropout))
  model.add(Dense(dataset.shape[1]))

  model.compile(loss='mse',
                optimizer=optimization,
                metrics=['mae', 'mse'])
  return model

In [ ]:
global activationFunction, optimizer
dimensions  = [Categorical(['tanh','sigmoid','linear','relu'], name='activationFunction'), #Search space, all parameters are nomminal
              Categorical(['RMSprop', 'Adam', 'SGD', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam'], name='optimizer')]
defact = ['tanh','sigmoid','linear','relu']
defopt =  ['RMSprop', 'Adam', 'SGD', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam']
default_parameters = [random.choice(defact), random.choice(defopt)]
@use_named_args(dimensions=dimensions) #Combine Objective function with its search space
def gp_minimize_opt_function(activationFunction, optimizer):

  global loss, best_loss, dataset, lookback, trainings
  epochs = int(round(x5))
  batch_size = int(round(x6))
  
  graph = tensorflow.Graph()
  with tensorflow.compat.v1.Session(graph=graph):

    model = build_model(activationFunction, optimizer)
    trainX, testX, trainY, testY = train_test(model, dataset, lookback)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[EarlyStopper]) 
    loss, _, _ = model.evaluate(testX, testY, verbose=2)

    if loss < best_loss:
      best_loss = loss
      model.save('Models/best')
      lb=testX.shape[1]
      print(lb, file=open('Models/lookback.txt','w'))
     
      with open("hyper.txt", "w") as f:
        var1 ="neurons = ",str(neurons)
        var2 = "layers = ",str(numOfLayers)
        var3 = "lookback = ",str(lookback)
        var4 ="dropout = ",str(dropout)
        var5 = "learning rate = ",str(x4)
        var6 = "epochs = ",str(epochs)
        var7 = "batch_size = ",str(batch_size)
        var8 = "number of lstm/conv1d layers = ",str(noOfLSTM_CONV)
        var9 = "number of pool size = ",str(pool_size)
        var10 = "activation function = ",activationFunction
        var11 = "optimizer = ",optimizer
        i = var1[0]+var1[1]+"||"+var2[0]+var2[1]+"||"+var3[0]+var3[1]+"||"+var4[0]+var4[1]+"||"+var5[0]+var5[1]+"||"+var6[0]+var6[1]+"||"+var7[0]+var7[1]+"||"+var8[0]+var8[1]+"||"+var9[0]+var9[1]+"||"+var10[0]+var10[1]+"||"+var11[0]+var11[1]   
        f.write(i)

  return loss

In [ ]:
def pso_opt_fun(x):
  global bn, bayes_inputs, bayes_results, bayes_time, bt, median_loss, best_swarm_loss, counter
  global x0, x1, x2, x3, x4, x5, x6, x7, x8
  x0 = x[0]
  x1 = x[1]
  x2 = x[2]
  x3 = x[3]
  x4 = x[4]
  x5 = x[5]
  x6 = x[6]
  x7 = x[7]
  x8 = x[8]
  calls = 3
  
  bt=time.time()
  if not bayes_inputs:
	  bayes = gp_minimize(func=gp_minimize_opt_function, dimensions=dimensions, acq_func='EI', 
                        n_calls=calls, n_random_starts=1, x0=default_parameters, model_queue_size=1)
  else:
	  bayes = gp_minimize(func=gp_minimize_opt_function, dimensions=dimensions, acq_func='EI', 
                        n_calls=calls, n_random_starts=1, x0=bayes_inputs, y0=bayes_results, model_queue_size=1)
  bayes_time=bayes_time+time.time()-bt
  bn=bn+1	
  bayes_inputs.extend(bayes.x_iters)
  bayes_results.extend(bayes.func_vals) 		
  keep=calls*bn
  bayes_inputs=bayes_inputs[-keep:]
  bayes_results=bayes_results[-keep:]
  bayes_inputs, bayes_results=remove_dupes(bayes_inputs, bayes_results)
  error=min(bayes_results[-calls:])
  
  return error

In [ ]:
EarlyStopper = EarlyStopping(patience=1, monitor='loss', mode='min')
trainings=0
best_loss = 1
defineModel = False
bn=0
bayes_inputs = []
bayes_results = []
time_total=[]
bayes_time_total=[]
bt=0
bayes_time=0

In [ ]:
#x0=neurons, x1=layers, x2=lookback
#x3=dropout, x4=learning rate, x5=epochs
#x6=batch_size, x7=number of lstm/conv1d layers, x8=pool_size
lb = [1, 1, 1, 0, 0.001, 20, 1, 1, 1]
ub = [128, 5, 5, 0.5, 0.2, 200, 1000, 2, 5]

In [ ]:
mydatasetfile = 'https://docs.google.com/uc?export=download&id=1XhbneHtO6R5b2XD401J6kcfccMdl0c1f'
dataset_name = mydatasetfile.split("/")[-1]
dataset, scaler = data_preparation(mydatasetfile)
dataframe = pd.read_csv(mydatasetfile, engine='python')

In [ ]:
start=time.time()
xopt, fopt = pso(pso_opt_fun, lb, ub, swarmsize=10, omega=1, phip=0.5, phig=1.0, maxiter=10, minstep=2, minfunc=0.00015)
mytime=time.time()-start

print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))
print(start, mytime)

In [ ]:
mymodel=keras.models.load_model('Models/best')
mydatasetfile = 'https://docs.google.com/uc?export=download&id=1XhbneHtO6R5b2XD401J6kcfccMdl0c1f'
dataset_name = mydatasetfile.split("/")[-1]
dataset, scaler = data_preparation(mydatasetfile)
text_file = open("Models/lookback.txt", "r")
lookback= int(text_file.read(1))
if hasLSTM(mymodel) or hasCONV1D(mymodel):
  dataset,datasetY=split_sequences(dataset,lookback,1)  
  datasetY=datasetY.reshape(datasetY.shape[0],datasetY.shape[2]) 
else:
  dataset,datasetY=split_sequences(dataset,1,1)  
  dataset=dataset.reshape(dataset.shape[0],dataset.shape[2])
  datasetY=datasetY.reshape(datasetY.shape[0],datasetY.shape[2])  
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
trainX, testX = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
trainY, testY = datasetY[0:train_size,:], datasetY[train_size:len(dataset),:]

In [ ]:
mse, mae, mse_ram, mae_ram, mse_cpu, mae_cpu = metrics(mymodel, testX, testY, scaler)
infS, infB = inference(mymodel, testX)

In [ ]:
mymodel.summary()
file_o=open("hyper.txt")   
content=file_o.read()                 
print(content)                       
file_o.close()

In [ ]:
import math  
print('PSO results using', dataset_name, 'as dataset')
print('Best mse: %.6f	mae: %.6f	rmse: %.6f	training time: %.0f s' % (mse, mae, math.sqrt(mse), mytime))
print('Single inference time: %.3f s	batch inference time: %.3f s ' % (infS, infB))
print('RAM mse: %.6f	RAM mae: %.6f	RAM rmse: %.6f' % (mse_ram, mae_ram, math.sqrt(mse_ram)))
print('CPU mse: %.6f	CPU mae: %.6f	CPU rmse: %.6f' % (mse_cpu, mae_cpu, math.sqrt(mse_cpu)))
print('DeepLearning,',mytime,',',math.sqrt(mse),',',mae,',',infS,',',infB,',',math.sqrt(mse_cpu),',',mae_cpu,',',math.sqrt(mse_ram),',',mae_ram, file=open('results.csv','a'))
